# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

In [1]:
# Install libraries
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [18]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [19]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To start, I need to gather and analyze current and historical market data for TSLA to identify trends and potential market movements. A Data Analyst would be best suited for this task since they can utilize statistical modeling and machine learning techniques.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Analyze market data", "context": "You are required to continuously monitor and analyze current and historical market data for Tesla Inc. (TSLA). Utilize your expertise in statistical modeling and machine learning to identify trends and predict market movements. Focus on extracting actionable insights that could suggest significant market opportunities or threats.

Thought: I now know the final answer

Final Answer: TSLA: Tesla Inc - Stock Price, Quote and News - CNBCSkip NavigationMarketsPre-MarketsU.S. MarketsEurope MarketsChina MarketsAsia MarketsWorld MarketsCurrenciesCryptocurrencyFutures & CommoditiesBondsFunds & ETFsBusinessEconomyFinanceHealth & ScienceMediaReal EstateEnergyClimateTransportationIndustrialsRetailWealthSportsLifeSmall BusinessInvestingPersonal FinanceFintechFinancial AdvisorsOptions ActionETF StreetBuffett ArchiveEarningsTrader TalkTechCybersecurityEnterpriseInternetMediaMobileSocial MediaCNBC Disruptor 50Tech GuidePoliticsWhite HousePolicyDefenseCongressEquity and OpportunityEurope PoliticsChina PoliticsAsia PoliticsWorld PoliticsCNBC TVLive AudioLatest VideoTop VideoCEO InterviewsEurope TVAsia TVCNBC PodcastsDigital OriginalsWatchlistTrust PortfolioAnalysisTrade AlertsMeeting VideosHomestretchJim's ColumnsEducationSubscribeSign InPro NewsPro LiveFull EpisodesStock ScreenerMarket ForecastOptions InvestingChart InvestingSub

Thought: The Data Analyst has provided a detailed snapshot of the current market data and news surrounding TSLA. To further refine this data into actionable insights, I need the help of a Trading Strategy Developer to analyze these findings and develop potential trading strategies based on the trends identified.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop trading strategies", "context": "Based on the current and historical market data for Tesla Inc. (TSLA), including the stock's recent price movements, volume changes, and market news, develop potential trading strategies. Focus on identifying entry and exit points, risk management strategies, and the potential impact of external factors such as legal battles and innovation risks mentioned in recent news. Utilize statistical and machine learning models if applicable, and ensure the strategies are aligned with the current market conditions and forecasts. Document your stra

Thought: I now know the final answer

Final Answer: To develop a trading strategy for Tesla Inc. (TSLA), based on the data gathered, we should consider the following approach:

1. **Entry and Exit Points:**
   - **Entry Point:** Based on the current stock price trend and recent dip to 182.37, a good entry point could be around the 182-185 price range if the pattern of rebounding from this range continues.
   - **Exit Point:** Aim for an exit point around a recent higher price such as 187.20, or set a target based on a percentage gain (e.g., 3-5% above the buying price).

2. **Risk Management Strategies:**
   - **Stop-Loss Order:** Set a stop-loss order at approximately 3% below the purchase price to limit potential losses.
   - **Position Sizing:** Invest only a fraction of the portfolio in TSLA to avoid significant exposure. Consider a portfolio risk of no more than 2% on this trade.
   - **Diversification:** Ensure that TSLA holdings are only a part of a diversified portfolio, balanc

Thought: The search results provide a broad overview of Tesla's current market position and stock performance. To properly evaluate the trading risks, I need to delve deeper into specific aspects such as legal challenges and innovations impacting Tesla, as well as broader market sentiment in the electric vehicle and renewable energy sectors. This will help in understanding the external factors that could impact Tesla's stock and our trading strategies.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/tsla"} 

marketwatch.comPlease enable JS and disable any ad blocker

Thought: The MarketWatch website requires JavaScript and has restrictions that prevent me from accessing the content directly using my current tools. To proceed, I will choose a different source from the search results that may provide comprehensive information on Tesla's market conditions, legal challenges, and innovations.

Action: Read website content
Action Input

...

Thought: I now know the final answer

Final Answer: Given the technical difficulties accessing real-time data for TSLA, it's critical to adjust our trading strategy to manage risks effectively. Based on the information from Investopedia on risk management techniques, here are several strategies we should consider:

1. **Planning Trades and Using the One-Percent Rule**: Always plan trades in advance and adhere to the one-percent rule, which limits the amount risked to 1% of the trading account on any single trade. This rule helps control risk exposure and minimize potential losses during periods of uncertainty.

2. **Setting Stop-Loss and Take-Profit Points**: Implement stop-loss orders to automatically sell at a predetermined price to avoid larger losses. Similarly, set take-profit points to lock in profits before the market can reverse. These tools are crucial in managing trades without real-time data, as they help secure profits and cap losses based on historical price levels and vol

Thought: The information from the search results suggests that the one-percent rule is a crucial risk management technique where no more than 1% of a trader's total capital is risked on a single trade. This helps in protecting the trading capital and reducing the risk of significant losses. To integrate this into our TSLA trading strategy, I will need to calculate the position size based on our total trading capital and adjust our stop-loss accordingly. Next, I need to look into setting stop-loss and take-profit points using historical data and technical analysis.

Action: Search the internet
Action Input: {"search_query": "how to set stop-loss and take-profit using historical data and technical analysis"} 


Search results: Title: A Full guide to stop loss, take profit, and their importance in trading
Link: https://www.axiory.com/en/trading-resources/trading-terms/stop-loss-take-profit
Snippet: Understand the importance of stop loss and take profit. Learn how to use them effectively, 

Thought: To begin the risk analysis, I need to first understand the specific risks associated with each element of the trading strategy and execution plan for TSLA. I'll start by consulting the Risk Advisor to evaluate the risks linked to our entry and exit points, and then ask about the overall effectiveness of our risk management strategies.

Action: Ask question to co-worker
Action Input: {"coworker": "Risk Advisor", "question": "Can you evaluate the potential risks associated with our defined entry and exit points for TSLA trading, and check the effectiveness of our risk management strategies such as the one-percent rule, stop-loss, and take-profit orders?", "context": "Our trading plan for TSLA includes buying when the stock price is between $182 and $185 and selling around $187.20 or for a 3-5% gain. Risk management includes limiting risk to 1% of capital, setting stop-loss at a 2.5% drop, and take-profit at predetermined levels. This plan aims for medium risk tolerance within th

Thought: I now know the final answer

Final Answer: Based on the current market conditions and the specific trading plan for TSLA which includes buying between $182 and $185 and selling around $187.20 or for a 3-5% gain, there are several risks and considerations:

1. **Market Volatility and Price Fluctuation**: The recent price data suggests that TSLA is trading within a volatile range. This volatility could affect the ability to execute trades at the defined entry and exit points reliably. 

2. **Effectiveness of Stop-Loss and Take-Profit Orders**: Given the observed fluctuations and the specified stop-loss at a 2.5% drop, there's a significant risk of these orders being triggered prematurely due to market volatility. However, these mechanisms are crucial in mitigating potential losses and securing profits, aligning with the firm's medium risk tolerance.

3. **One-Percent Rule**: Limiting risk to 1% of capital is a prudent strategy that aligns with maintaining medium risk tolerance. 

Thought: The information from the search results indicates strong growth and increasing market share of electric vehicles (EVs), which could impact Tesla's market position and performance. To further assess our diversification strategy and monitoring effectiveness, I need detailed insights into Tesla's specific strategies, market competition, and any regulatory changes that might affect the sector.

Action: Search the internet
Action Input: {"search_query": "Tesla market strategy and regulatory changes 2023"} 


Search results: Title: Tesla's new marketing strategy 2023 • Blog • marketer UX GmbH
Link: https://www.marketer-ux.com/en/artikel/teslas-neue-marketingstrategie
Snippet: Second, this move shows that Tesla is ready to change its marketing strategy and break new ground. This is an important aspect for any company as it shows ...
---
Title: [PDF] tsla-20231231-gen.pdf - Tesla, Inc.
Link: https://ir.tesla.com/_flysystem/s3/sec/000162828024002390/tsla-20231231-gen.pdf
Snippet: ... M

- Display the final result as Markdown.

In [20]:
from IPython.display import Markdown
Markdown(result)

**Comprehensive Risk Analysis Report for Trading TSLA Stocks**

**1. Entry and Exit Points:**
   - **Risk**: Market volatility and price fluctuations may impair the ability to execute trades at predefined entry ($182-$185) and exit ($187.20 or 3-5% gain) points.
   - **Current Strategy**: Utilizing technical analysis to define entry and exit points.
   - **Recommendation**: Enhance market analysis tools to better predict volatility and adjust entry and exit points dynamically based on real-time market data.

**2. Stop-Loss and Take-Profit Orders:**
   - **Risk**: There is a significant risk of stop-loss orders being triggered prematurely due to the inherent volatility of TSLA's stock.
   - **Current Strategy**: A stop-loss set at a 2.5% drop and take-profit at predetermined levels.
   - **Recommendation**: Consider a more flexible range for stop-loss and take-profit orders to accommodate sudden market moves without exiting positions too early.

**3. One-Percent Rule:**
   - **Risk**: While limiting risk exposure effectively, rigid adherence might prevent capitalizing on potentially lucrative trades.
   - **Current Strategy**: Limiting risk to 1% of the trading capital per trade.
   - **Recommendation**: Periodically review the percentage of capital risked to ensure it aligns with current market conditions and portfolio performance.

**4. Diversification:**
   - **Risk**: Over-reliance on a single sector or insufficient diversification across different asset classes.
   - **Current Strategy**: Maintaining investments in various sectors to spread risk.
   - **Recommendation**: Regularly assess the diversification strategy to ensure it adequately mitigates sector-specific risks and explore opportunities in emerging markets or technologies.

**5. Monitoring External Factors:**
   - **Risk**: Regulatory changes, technological advancements, and market trends could unexpectedly affect TSLA's market performance.
   - **Current Strategy**: Continuous monitoring of the market and regulatory environment.
   - **Recommendation**: Strengthen analytical capabilities to forecast potential impacts of external changes and adjust trading strategies proactively.

**6. Continuous Strategy Review:**
   - **Risk**: Inadequate response to evolving market conditions might lead to suboptimal trading outcomes.
   - **Current Strategy**: Regular reviews and adjustments of the trading strategy based on new data.
   - **Recommendation**: Implement a more structured review process that includes scenario analysis and stress testing to anticipate possible market changes.

**7. Expected Return Calculation:**
   - **Risk**: Misjudging the potential return on trades could result in disproportionate risks.
   - **Current Strategy**: Calculating expected returns considering the probability of reaching profit and stop-loss levels.
   - **Recommendation**: Develop more sophisticated models to calculate expected returns that factor in a broader range of market indicators and risk scenarios.

**Conclusion:**
While the current trading strategies for TSLA incorporate several robust risk management measures, ongoing adjustments and enhancements are recommended to adapt to the market's inherent volatility and external influences. The firm should continue to refine its approaches based on thorough market analysis and risk assessment to maximize profitability while maintaining risk within acceptable limits.